In [76]:
import numpy as np
import pandas as pd
import sys

import csv
from scipy.stats import skew, kurtosis
from numpy import nan
import math


In [77]:

import pyeeg as p

In [78]:
pip install mne


Note: you may need to restart the kernel to use updated packages.


In [79]:
import mne

In [80]:
def all_psd(data):
    fs = 256                                
    N = data.shape[1] # total num of points 

    # Get only in postive frequencies
    fft_vals = np.absolute(np.fft.rfft(data))

    n_rows = fft_vals.shape[0]
    n_cols = fft_vals.shape[1]
    psd_vals = np.zeros(shape=(n_rows, n_cols))

    for i in range(n_rows):
        for j in range(n_cols):
            psd_vals[i][j] = (N/fs) * fft_vals[i][j] * fft_vals[i][j];


    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(data.shape[1], 1.0/fs)

    # Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band
    eeg_band_fft = dict()
    psd_vals_list = []
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = psd_vals[:,freq_ix]
        for i in range(len(eeg_band_fft[band])): 
            psd_vals_list.append(np.mean(eeg_band_fft[band][i]) * 1000000)
    return psd_vals_list

In [99]:
seizure_var=1
def add_row(ip_psd, start, end):
    row_to_add = []
    
    ip_psd = ip_psd[:,start:end]
    ip_psd = ip_psd[:][0]
    
    
    p = all_psd(ip_psd)
    p.append(seizure_var)
    
    return p

In [100]:
data = mne.io.read_raw_edf('chb01_03.edf')
header = ','.join(data.ch_names)
df = pd.DataFrame(data[:][0])
df = df.transpose()

Extracting EDF parameters from C:\Users\AAI\Desktop\btp\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-100-e0b81f09e547>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf('chb01_03.edf')


In [102]:
start = temp = 2996*256 
duration = 10*256
end = 3036*256

df1 = df.iloc[start:end, :]
df1 = pd.DataFrame(df1)
df1.shape

(10240, 23)

In [103]:

res = pd.DataFrame()

#first iteration run in 'w' mode, all subsequent iteration run in 'a' mode
#with open('test.csv', 'w') as file:
#    writer = csv.writer(file)
while temp < end:    
    row = add_row(data, temp, temp + duration)
    res=res.append(pd.Series(row),ignore_index=True)
#        writer.writerow(row)
    temp += duration

#res.columns = ['Variance', 'Kurtosis', 'Skewness', 'Petrosian Fractal Dimension', 'Hjorth Mobility', 'Hjorth Complexity', 'Spectral Entropy', 'Label']
res

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,2398.430145,1057.376321,1228.272867,1425.589491,4015.352730,2253.893527,1467.410656,2859.182774,4128.874769,5727.598745,...,32.702493,5.868905,1.389389,1.267681,1.011791,5.608958,28.365478,28.583358,32.702493,1.0
1,5683.760177,3870.849233,1800.316498,955.461104,6227.326422,4427.130113,2142.612872,2960.410795,5583.169827,6398.922857,...,143.669691,11.165056,5.857666,1.767788,2.342996,6.234717,42.395905,79.530479,143.669691,1.0
2,15883.536738,6928.386027,3643.387289,1681.489361,17768.640030,7031.835493,2702.708040,3313.457312,8903.371842,10209.192165,...,207.652680,15.822586,4.673614,1.478758,18.882396,20.171697,30.897911,101.522740,207.652680,1.0
3,12425.395306,4175.535057,2614.413841,2501.161790,15171.964415,5781.670067,3326.714216,3808.665648,9489.501332,9354.724767,...,139.274665,19.535967,2.236787,1.314111,3.419047,21.297453,67.694015,101.309500,139.274665,1.0


In [85]:
df_final = pd.DataFrame()

In [104]:
df_final = df_final.append(res)

In [105]:
df_final.shape

(104, 116)

In [114]:
X = df_final.iloc[:,0:115].values
#print(X);
y = df_final.iloc[:,-1].values
print(y);

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1.]


In [115]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [116]:
rf_cs = RandomForestClassifier(n_jobs=-1,random_state=0)

In [117]:
param_grid = { 'n_estimators': [10,50,100,200,250,500,700],}
    
CV_rfc = GridSearchCV(estimator=rf_cs, param_grid=param_grid, cv= 10)
CV_rfc.fit(X, y)
print (CV_rfc.best_params_)
op_RF = CV_rfc.best_estimator_
op_RF.fit(X,y)

C:\Users\AAI\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


{'n_estimators': 50}


RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=0)

In [118]:
pip install boruta

In [119]:
from boruta import BorutaPy



In [120]:
# define Boruta feature selection method
feat_selector = BorutaPy(op_RF, n_estimators='auto', verbose=2, random_state=1)

In [121]:
feat_selector.fit(X, y)


Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	115
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	115
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	115
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	115
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	115
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	115
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	115
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	90
Rejected: 	25
Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	90
Rejected: 	25
Iteration: 	10 / 100
Confirmed: 	0
Tentative: 	90
Rejected: 	25
Iteration: 	11 / 100
Confirmed: 	1
Tentative: 	89
Rejected: 	25
Iteration: 	12 / 100
Confirmed: 	1
Tentative: 	89
Rejected: 	25
Iteration: 	13 / 100
Confirmed: 	1
Tentative: 	89
Rejected: 	25
Iteration: 	14 / 100
Confirmed: 	2
Tentative: 	86
Rejected: 	27
Iteration: 	15 / 100
Confirmed: 	2
Tentative: 	86
Rejected: 	27
Iteration: 	16 / 100
Confirmed: 	2
Tentative: 	86

BorutaPy(estimator=RandomForestClassifier(n_estimators=124, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x27C1B4A4B40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x27C1B4A4B40, verbose=2)

In [122]:
feat_selector.support_

array([False, False,  True, False,  True,  True, False,  True, False,
       False,  True,  True, False, False, False,  True,  True,  True,
        True,  True, False,  True,  True,  True, False, False, False,
        True, False, False, False,  True,  True,  True,  True, False,
       False,  True,  True, False, False,  True,  True,  True,  True,
        True, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True, False, False, False, False, False,  True,
       False,  True, False, False, False, False, False, False, False,
        True, False,  True, False,  True, False, False,  True,  True,
        True,  True, False, False,  True,  True, False])

In [123]:
np.size(feat_selector.support_) 

115

In [124]:
feat_selector.ranking_

array([ 2,  2,  1,  2,  1,  1,  2,  1,  2,  2,  1,  1,  2,  2,  2,  1,  1,
        1,  1,  1,  2,  1,  1,  1,  2,  2, 10,  1,  2,  2, 14,  1,  1,  1,
        1,  2,  2,  1,  1,  2,  8,  1,  1,  1,  1,  1, 32,  9,  1, 32, 32,
       32, 20, 20, 32, 32, 11, 32, 32,  7,  2, 18, 32, 32,  1,  2,  1,  1,
        2,  3,  3, 32, 12, 20, 32, 32, 32, 16, 32, 32, 32,  1,  1,  1,  5,
       32, 32, 32, 14,  1,  2,  1,  5, 13, 17,  2,  2, 32,  2,  1,  2,  1,
        2,  1,  2,  2,  1,  1,  1,  1,  2,  3,  1,  1,  2])

In [125]:
X_filtered = feat_selector.transform(X)

In [134]:
#X_filtered

In [131]:
#np.size(X_filtered)

In [132]:
test=pd.DataFrame(X_filtered)
#test

In [133]:
#test.shape